# Compute FLOPS

In [1]:
import torch
import numpy as np
import sys
sys.path.append('../')

from model_utils import load_model

# MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'
# MODEL_NAME = 'faster_rcnn_vgg16'

sys.path.append('/workspace/home/jgusak/maxvol_objects/facebook_frcnn/')
import maskrcnn_benchmark
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_resnet50_fpn'

model = load_model(MODEL_NAME)
# model = torch.load('/workspace/raid/data/eponomarev/results/musco_voc/vgg16_fasterrcnn/tucker2/rank_selection:nx/ranks:/1.77x_layer_groups:1/vgg-16.bb')

# compressed_model =  torch.load('/workspace/raid/data/jgusak/backbone_iter_0-0.pth')
# compressed_model = torch.load('/workspace/raid/data/eponomarev/results/musco_voc/vgg16_fasterrcnn/tucker2/rank_selection:nx/ranks:/1.77x_layer_groups:1/backbone_iter:0-1.pth')
# compressed_model = torch.load("/workspace/raid/data/eponomarev/frcnn_models_hachiko2/resnet50_fasterrcnn/cp3/STEP_INV_LBL3_cp3_5/vbmf_0.9/0/2/compressed_frcnn_model.backbone")
# compressed_model = torch.load("/workspace/raid/data/eponomarev/frcnn_models_hachiko2/resnet50_fasterrcnn/cp3/STEP_INV_LBL3_cp3_10/vbmf_0.9/0/2/compressed_frcnn_model_backbone.pth")

model

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Sequential(
  (body): ResNet(
    (stem): StemWithFixedBatchNorm(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
    )
    (layer1): Sequential(
      (0): BottleneckWithFixedBatchNorm(
        (downsample): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): FrozenBatchNorm2d()
        )
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (conv2): Sequential(
          (conv2-0): Conv2d(64, 14, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (conv2-1): Conv2d(14, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (conv2-2): Conv2d(14, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (bn2): FrozenBatchNorm2d()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): FrozenBatchNorm2d()
      )
   

In [2]:
compressed_model = model

In [3]:
import sys
sys.path.append("../")

from flopco import FlopCo

In [4]:
flopco_m = FlopCo(model)
flopco_cm = FlopCo(compressed_model)

In [6]:
flopco_m.total_flops / flopco_cm.total_flops

1.0